LEVEL: determining the level of the user by looking at the terminology it knows.

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

# autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

LLM models:  ['gpt-4-32k', 'gpt-4']


In [5]:
system_message = """We are playing a simulation game. 
You are assessing the level of a simulated user
that only knows the terminology given and nothing more. You need to assess the level of the simulated user on the topic given.
The topic will be given after TOPIC.
The known terminology will be given after TERMINOLOGY.
"""

# 1. create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant", 
    system_message=system_message,
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
    is_termination_msg=termination_msg
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.AssistantAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    system_message="""You always answer with TOPIC: topic; TERMINOLOGY: list of terminology.""",
    max_consecutive_auto_reply=0,
    is_termination_msg=termination_msg,
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)
user_message = """TOPIC: machine learning
TERMINOLOGY: {
    "terminology": [
        "down", "under", "over", "between",
        "neural network", "deep learning", "machine learning"
    ]
}"""
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message=user_message,
)

user_proxy (to assistant):

TOPIC: machine learning
TERMINOLOGY: {
    "terminology": [
        "down", "under", "over", "between",
        "neural network", "deep learning", "machine learning"
    ]
}

--------------------------------------------------------------------------------
assistant (to user_proxy):

The simulated user seems to have a beginner's knowledge level on the topic of machine learning. The terminologies mentioned only include basic concepts and do not include any advanced terms related to this subject matter.

--------------------------------------------------------------------------------
